# Mapping out MPAs, bioregions, and the sampling sites

In [1]:
# import libraries
import os
import salem
import warnings

import geopandas as gpd
import shapefile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from json import dumps

import folium
from folium.plugins import MarkerCluster

import numpy as np
import pandas as pd
import xarray as xr


In [2]:
# load shapefiles for bioregion and MPA polygons

marine_sites = gpd.GeoDataFrame.from_file("Data/Shapefiles/sampling_sites/marine_sampling_sites.shp")
pisco_sites = gpd.GeoDataFrame.from_file("Data/Shapefiles/sampling_sites/pisco_sampling_sites.shp")
ccfrp_sites = gpd.GeoDataFrame.from_file("Data/Shapefiles/sampling_sites/ccfrp_sampling_sites.shp")

# split up into MPA vs reference sites
marine_mpa_sites = marine_sites[marine_sites['mpa_design'] != 'reference'] 
marine_reference_sites = marine_sites[marine_sites['mpa_design'] == 'reference'] 

pisco_mpa_sites = pisco_sites[pisco_sites['reserve_st'] == 'MPA'] 
pisco_reference_sites = pisco_sites[pisco_sites['reserve_st'] == 'REF'] 


ccfrp_mpa_sites = ccfrp_sites[ccfrp_sites['reserve_st'] == 'MPA'] 
ccfrp_reference_sites = ccfrp_sites[ccfrp_sites['reserve_st'] == 'REF']

In [3]:
# define plotting styles for the region/MPA polygons

eez_style = {'fillOpacity': 0.5, 'fillColor': '#33ccff', 'color': '#0066ff', 'weight': 0.75}
state_style = {'fillOpacity': 0.5, 'fillColor': '#0066ff', 'color': '#0000cc', 'weight': 0.75}
mpa_style = {'fillOpacity': 0.5, 'fillColor': '#cc66ff', 'color': '#550080', 'weight': 0.75}



In [4]:
# create base map

mpa_map = folium.Map(location = [36.897, -121.923], zoom_start = 6, tiles = 'Stamen Terrain')


In [5]:
# add GeoJSONs for region/MPA polygons

folium.GeoJson("Data/Shapefiles/eez_bioregions/ca_eez_all.geojson",
               name = "eez_bioregions",
               style_function = lambda x: eez_style,
              ).add_to(mpa_map)

folium.GeoJson("Data/Shapefiles/state_waters_bioregions/ca_state_waters.geojson",
               name = "state_bioregions",
               style_function = lambda x: state_style
              ).add_to(mpa_map)

folium.GeoJson("Data/Shapefiles/mpas/all_mpas/ca_all_mpas.geojson",
               name = "mpas",
               style_function = lambda x: mpa_style
              ).add_to(mpa_map)



# GeoJson(geojson_data, popup=folium.GeoJsonPopup(fields=['NAME']))



In [6]:
#  

# MARINe sites
## MPAs 
for longitude, latitude, marine_sit, start_date, end_date, number_sam in marine_mpa_sites[['longitude','latitude','marine_sit','start_date','end_date','number_sam']].values:
    folium.Marker(location=[latitude,longitude],
                  popup = folium.Popup('Program: MARINe <br> Site Name: {} <br> Status: MPA <br> Start Date: {} <br> End Date: {} <br> Number of Surveys: {}'
                                       .format(marine_sit, start_date, end_date, number_sam),
                                       max_width=450,min_width=450), 
                  icon = folium.Icon(color = 'red',
                                     icon = 'star',
                                     prefix = 'fa'),
                  clustered_marker = True
                  ).add_to(mpa_map)

# Reference sites
for longitude, latitude, marine_sit, start_date, end_date, number_sam in marine_reference_sites[['longitude','latitude','marine_sit','start_date','end_date','number_sam']].values:
    folium.Marker(location=[latitude,longitude],
                  popup = folium.Popup('Program: MARINe <br> Site Name: {} <br> Status: REF <br> Start Date: {} <br> End Date: {} <br> Number of Surveys: {}'
                                       .format(marine_sit, start_date, end_date, number_sam),
                                       max_width=450,min_width=450), 
                  icon = folium.Icon(color = 'red',
                                     icon = 'circle',
                                     prefix = 'fa'),
                  clustered_marker = True
                  ).add_to(mpa_map)
    

    
# PISCO sites
# MPAs
for longitude, latitude, program, site_name, start_year, end_year, number_sam in pisco_mpa_sites[['longitude','latitude','program','site_name','start_year','end_year','number_sam']].values:
    folium.Marker(location=[latitude,longitude],
                  popup = folium.Popup('Program: PISCO/{} <br> Site Name: {} <br> Status: MPA <br> Start Year: {} <br> End Year: {} <br> Number of Surveys: {}'
                                       .format(program, site_name, start_year, end_year, number_sam),
                                       max_width=450,min_width=450), 
                  icon = folium.Icon(color = 'darkblue',
                                     icon = 'star',
                                     prefix = 'fa'
                                     ),
                  clustered_marker = True
                 ).add_to(mpa_map)

    
# Reference sites
for longitude, latitude, program, site_name, start_year, end_year, number_sam  in pisco_reference_sites[['longitude','latitude','program','site_name','start_year','end_year','number_sam']].values:
    folium.Marker(location=[latitude,longitude],
                  popup = folium.Popup('Program: PISCO/{} <br> Site Name: {} <br> Status: REF <br> Start Year: {} <br> End Year: {} <br> Number of Surveys: {}'
                                       .format(program, site_name, start_year, end_year, number_sam),
                                       max_width=450,min_width=450), 
                  icon = folium.Icon(color = 'darkblue',
                                     icon = 'circle',
                                     prefix = 'fa'
                                     ),
                  clustered_marker = True
                 ).add_to(mpa_map)


    
# CCFRP sites
## MPAs
for longitude, latitude, site_name, start_year, end_year, number_sam in ccfrp_mpa_sites[['longitude','latitude','site_name','start_year','end_year','number_sam']].values:
    folium.Marker(location=[latitude,longitude],
                  popup = folium.Popup('Program: CCFRP <br> Site Name: {} <br> Status: MPA <br> Start Year: {} <br> End Year: {} <br> Number of Surveys: {}'
                                       .format(site_name, start_year, end_year, number_sam),
                                       max_width=450,min_width=450), 
                  icon = folium.Icon(color = 'darkpurple',
                                     icon = 'star',
                                     prefix = 'fa'),
                  clustered_marker = True
                 ).add_to(mpa_map)
    
## Reference sites
for longitude, latitude, site_name, start_year, end_year, number_sam in ccfrp_reference_sites[['longitude','latitude','site_name','start_year','end_year','number_sam']].values:
    folium.Marker(location=[latitude,longitude],
                  popup = folium.Popup('Program: CCFRP <br> Site Name: {} <br> Status: REF <br> Start Year: {} <br> End Year: {} <br> Number of Surveys: {}'
                                       .format(site_name, start_year, end_year, number_sam), 
                                       max_width=450,min_width=450), 
                  icon = folium.Icon(color = 'darkpurple',
                                     icon = 'circle',
                                     prefix = 'fa'),
                  clustered_marker = True
                 ).add_to(mpa_map)
    

In [7]:
# mpa_map

mpa_map.save("Monitoring_Site_Map.html")